# PANDA EfficientNet-B0 Baseline with 36 x tiles_256

Hi everyone,

I'm here to show you how to train a single efficientnet-b0 model to get LB 0.87

This is inference kernel and the Training kernel is avalilable here: https://www.kaggle.com/haqishen/train-efficientnet-b0-w-36-tiles-256-lb0-87



# TTA
    
Tile extraction start from different point (by adding more white padding)

In [1]:
DEBUG = False

In [2]:
import os
import sys
sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

In [3]:
import skimage.io
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from efficientnet_pytorch import model as enet

import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm


In [4]:
data_dir = '../input/prostate-cancer-grade-assessment'
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
df_sub = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

model_dir = '../input/panda-public-models'
image_folder = os.path.join(data_dir, 'test_images')
is_test = os.path.exists(image_folder)  # IF test_images is not exists, we will use some train images.
image_folder = image_folder if is_test else os.path.join(data_dir, 'train_images')

df = df_test if is_test else df_train.loc[:100]

tile_size = 256
image_size = 256
n_tiles = 36
batch_size = 8
num_workers = 1

device = torch.device('cuda')

print(image_folder)

../input/prostate-cancer-grade-assessment/train_images


# Model

In [5]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()

    def extract(self, x):
        return self.enet(x)
    
    def extract_features(self, x):
        f = self.enet.extract_features(x)
        return f

    def forward(self, x):
        x = self.extract(x)
        x = self.myfc(x)
        return x
    
    
def load_models(model_files):
    models = []
    for model_f in model_files:
        model_f = os.path.join(model_dir, model_f)
        backbone = 'efficientnet-b0'
        model = enetv2(backbone, out_dim=5)
        model.load_state_dict(torch.load(model_f, map_location=lambda storage, loc: storage), strict=True)
        model.eval()
        model.to(device)
        models.append(model)
        print(f'{model_f} loaded!')
    return models


model_files = [
    'cls_effnet_b0_Rand36r36tiles256_big_bce_lr0.3_augx2_30epo_model_fold0.pth'
]

models = load_models(model_files)

../input/panda-public-models/cls_effnet_b0_Rand36r36tiles256_big_bce_lr0.3_augx2_30epo_model_fold0.pth loaded!


In [6]:
import pickle
lgbm = pickle.load(open("../input/kn-models/trained_lgbm.pkl", 'rb'))
lgbm

# Dataset

In [7]:
def get_tiles(img, mode=0):
        result = []
        h, w, c = img.shape
        pad_h = (tile_size - h % tile_size) % tile_size + ((tile_size * mode) // 2)
        pad_w = (tile_size - w % tile_size) % tile_size + ((tile_size * mode) // 2)

        img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], constant_values=255)
        img3 = img2.reshape(
            img2.shape[0] // tile_size,
            tile_size,
            img2.shape[1] // tile_size,
            tile_size,
            3
        )

        img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
        n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
        if len(img) < n_tiles:
            img3 = np.pad(img3,[[0,N-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
        idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
        img3 = img3[idxs]
        for i in range(len(img3)):
            result.append({'img':img3[i], 'idx':i})
        return result, n_tiles_with_info >= n_tiles


class PANDADataset(Dataset):
    def __init__(self,
                 df,
                 image_size,
                 n_tiles=n_tiles,
                 tile_mode=0,
                 rand=False,
                 sub_imgs=False
                ):

        self.df = df.reset_index(drop=True)
        self.image_size = image_size
        self.n_tiles = n_tiles
        self.tile_mode = tile_mode
        self.rand = rand
        self.sub_imgs = sub_imgs

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        
        tiff_file = os.path.join(image_folder, f'{img_id}.tiff')
        image = skimage.io.MultiImage(tiff_file)[1]
        tiles, OK = get_tiles(image, self.tile_mode)

        if self.rand:
            idxes = np.random.choice(list(range(self.n_tiles)), self.n_tiles, replace=False)
        else:
            idxes = list(range(self.n_tiles))
        idxes = np.asarray(idxes) + self.n_tiles if self.sub_imgs else idxes

        n_row_tiles = int(np.sqrt(self.n_tiles))
        images = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
    
                if len(tiles) > idxes[i]:
                    this_img = tiles[idxes[i]]['img']
                else:
                    this_img = np.ones((self.image_size, self.image_size, 3)).astype(np.uint8) * 255
                this_img = 255 - this_img
                h1 = h * image_size
                w1 = w * image_size
                images[h1:h1+image_size, w1:w1+image_size] = this_img

#         images = 255 - images
        images = images.astype(np.float32)
        images /= 255
        images = images.transpose(2, 0, 1)

        return torch.tensor(images)


# Prediction

In [8]:
dataset = PANDADataset(df, image_size, n_tiles, 0)  # mode == 0
loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

dataset2 = PANDADataset(df, image_size, n_tiles, 2)  # mode == 2
loader2 = DataLoader(dataset2, batch_size=batch_size, num_workers=num_workers, shuffle=False)

In [9]:
LOGITS = []
LOGITS2 = []
mid_test_vals = []
#mid_test_vals2 = []
with torch.no_grad():
    for data in tqdm(loader):
        data = data.to(device)
        logits = models[0](data)
        LOGITS.append(logits)
        bb = models[0].extract_features(data).cpu().detach().numpy()
        mid_test_vals.append([bb])
    for data in tqdm(loader2):
        data = data.to(device)
        logits = models[0](data)
        LOGITS2.append(logits)
        #bb = models[0].extract_features(data).cpu().detach().numpy()
        #mid_test_vals2.append([bb])
LOGITS = (torch.cat(LOGITS).sigmoid().cpu() + torch.cat(LOGITS2).sigmoid().cpu()) / 2
PREDS = LOGITS.sum(1).round().numpy()
#mid_test_vals = (np.array(mid_test_vals) + np.array(mid_test_vals2)) / 2



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [10]:

len(mid_test_vals[8][0][0][0][0])

48

In [11]:
test_X = np.array([])

for i in range(len(mid_test_vals)):
    t1 = np.array([])
    t2 = np.array([])
    t3 = np.array([])
    t4 = np.array([])
    t5 = np.array([])
    t6 = np.array([])
    t7 = np.array([])
    t8 = np.array([])
    for k in range(len(mid_test_vals[i][0][0])):
        l1 = np.array([])
        l2 = np.array([])
        l3 = np.array([])
        l4 = np.array([])
        l5 = np.array([])
        l6 = np.array([])
        l7 = np.array([])
        l8 = np.array([])
        for j in range(len(mid_test_vals[i][0][0][0][0])):
            s1 = np.mean(mid_test_vals[i][0][0][k][j])
            s2 = np.mean(mid_test_vals[i][0][1][k][j])
            s3 = np.mean(mid_test_vals[i][0][2][k][j])
            s4 = np.mean(mid_test_vals[i][0][3][k][j])
            s5 = np.mean(mid_test_vals[i][0][4][k][j])
            s6 = np.mean(mid_test_vals[i][0][5][k][j])
            s7 = np.mean(mid_test_vals[i][0][6][k][j])
            s8 = np.mean(mid_test_vals[i][0][7][k][j])
            l1 = np.append(l1, s1)
            l2 = np.append(l2, s2)
            l3 = np.append(l3, s3)
            l4 = np.append(l4, s4)
            l5 = np.append(l5, s5)
            l6 = np.append(l6, s6)
            l7 = np.append(l7, s7)
            l8 = np.append(l8, s8)
        t1 = np.append(t1, l1.mean())
        t2 = np.append(t2, l2.mean())  
        t3 = np.append(t3, l3.mean())
        t4 = np.append(t4, l4.mean())  
        t5 = np.append(t5, l5.mean())
        t6 = np.append(t6, l6.mean())  
        t7 = np.append(t7, l7.mean())
        t8 = np.append(t8, l8.mean())  
    #print(t1.shape)
    test_X = np.append(train_X, t1.reshape(1, len(t1)))
    test_X = np.append(train_X, t2.reshape(1, len(t2)))
    test_X = np.append(train_X, t3.reshape(1, len(t3)))
    test_X = np.append(train_X, t4.reshape(1, len(t4)))
    test_X = np.append(train_X, t5.reshape(1, len(t5)))
    test_X = np.append(train_X, t6.reshape(1, len(t6)))
    test_X = np.append(train_X, t7.reshape(1, len(t7)))
    test_X = np.append(train_X, t8.reshape(1, len(t8)))
    #train_y = np.append(train_y, mid_train_vals[i][1][0].cpu().numpy().sum())
    #train_y = np.append(train_y, mid_train_vals[i][1][1].cpu().numpy().sum())

test_X = test_X.reshape(len(df), len(t1))
print(test_X.shape)
test_X = pd.DataFrame(test_X)
pred_lgbm = lgbm.predict(test_X)

NameError: name 'train_X' is not defined

In [12]:
PREDS = 0.7*PREDS + 0.3*pred_lgbm

NameError: name 'pred_lgbm' is not defined

In [13]:
df['isup_grade'] = PREDS.astype(int)
df[['image_id', 'isup_grade']].to_csv('submission.csv', index=False)
print(df.head())
print()
print(df.isup_grade.value_counts())
df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


                           image_id data_provider  isup_grade gleason_score
0  0005f7aaab2800f6170c399693a96917    karolinska           0           0+0
1  000920ad0b612851f8e01bcc880d9b3d    karolinska           0           0+0
2  0018ae58b01bdadc8e347995b69f99aa       radboud           4           4+4
3  001c62abd11fa4b57bf7a6c603a11bb9    karolinska           4           4+4
4  001d865e65ef5d2579c190a0e0350d8f    karolinska           0           0+0

1    31
0    20
4    18
2    12
3    11
5     9
Name: isup_grade, dtype: int64


,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0
...,...,...,...,...
96,027c4a5b5f9f1be80602ef51f2619bc2,karolinska,1,3+3
97,02805e3e20ef629ffeeee1e68cf24ab0,radboud,4,4+4
98,028098c36eb49a8c6aa6e76e365dd055,karolinska,0,0+0
99,0280f8b612771801229e2dde52371141,radboud,1,3+3
